# Assignment 3
# CSCI-P556, Spring 2019
## Due date: Monday, March 25, 2019, by 11:59PM.

## Problem 1: Building a Neural Networks using Tensorflow's Low-Level API

The most popular deep learning library is Tensorflow. This library offers two ways develop deep learning models, these are through the High-Level API (Keras) and the Low-Level API. Keras' API is really straight-forward, below we include [Tensorflow's example](https://www.tensorflow.org/tutorials/):

In [0]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

In [0]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
'''
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)
'''
import os
import tarfile
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf

tfile=tarfile.open('kkanji.tar')
tfile.extractall('')


Easy, right? Well, we are going to get our hands dirty with the low-level API. **Your task for this problem is to implement a fully-connected deep neural network using [Tensorflow's low-level API](https://www.tensorflow.org/guide).** Make sure that you code can handle passing parameters for the number of layers and hidden units, as well as any other parameter that you deem necessary.

Note: we will take into account your models' accuracy when we grade. For example, if the average accuracy in the class is 91% accuracy and your model is giving 70% accuracy, then you did something wrong. If you aren't getting a good accuracy, check if your loss function is the correct one or try tweaking the hyperparameters.

In [39]:
!git clone https://github.com/rois-codh/kmnist
!python "kmnist//download_data.py"


fatal: destination path 'kmnist' already exists and is not an empty directory.
Please select a download option:
1) Kuzushiji-MNIST (10 classes, 28x28, 70k examples)
2) Kuzushiji-49 (49 classes, 28x28, 270k examples)
3) Kuzushiji-Kanji (3832 classes, 64x64, 140k examples)
> 2
Please select a download option:
1) NumPy data format (.npz)
> 1
100% 64569/64569 [00:09<00:00, 6481.64KB/s]
100% 161/161 [00:00<00:00, 258.89KB/s]
100% 10715/10715 [00:02<00:00, 3582.95KB/s]
100% 27/27 [00:00<00:00, 172.63KB/s]
All dataset files downloaded!


We will be using two versions of [Kuzushiji dataset](https://arxiv.org/abs/1812.01718) to test your models: Kuzushiji-49 and Kuzushiji-Kanji. The descriptions of datasets are:

> "Kuzushiji-49, as the name suggests, has 49 classes (28x28 grayscale, 270,912 images), is a much larger, but imbalanced dataset containing 48 Hiragana characters and one Hiragana iteration mark."

> "Kuzushiji-Kanji is an imbalanced dataset of total 3832 Kanji characters (64x64 grayscale, 140,426 images), ranging from 1,766 examples to only a single example per class."

If you read the description carefully, you can notice that these aren't datasets that we can just feed to the model without doing some pre-processing. Make sure that your code can handle this issue. Also, the number of layers and hidden units, as well as the hyperparameters are up to you to figure out. 

Instructions on how to download these datasets are available in their [Github page](https://github.com/rois-codh/kmnist). To clone Kuzushiji repository from Github to Google Colab, use:

!git clone https://github.com/rois-codh/kmnist

Note: all Linux commands are available in Google Colab by preceding them with a "!", just like we did above.

In [0]:
import os
import tarfile
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.utils import class_weight
    
def weight_bias_calc(p1,p2):
    weights =tf.Variable(tf.random_normal([p1,p2]))
    bias = tf.Variable(tf.random_normal([p2]))
    return weights,bias

def NN_model(x,data):
    for i in range(0,(len(data)-1)):
        w,b = weight_bias_calc(data[i],data[i+1])
        layer = tf.add(tf.matmul(x, w),b)
        if(0< i <len(data)-2):
            layer = tf.nn.relu(layer)
        x = layer
    return layer

def train_neural_network(x,y,NN_layers_units,iterations,classes,train_images, train_labels, test_images, test_labels, batch_size,class_weights):
    batch_x = None
    batch_y = None
    prediction = NN_model(x,NN_layers_units)
    scaled_logits = tf.multiply(prediction,class_weights)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=scaled_logits, labels=y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for itr in range(iterations):
            loss = 0
            for n in range(int(len(train_images) / batch_size)):
                if(n == int(len(train_images) / batch_size)-1):
                    batch_x = train_images[n*batch_size:]
                    batch_y = train_labels[n*batch_size:]
                else:
                    batch_x = train_images[n*batch_size:(n+1)*batch_size]
                    batch_y = train_labels[n*batch_size:(n+1)*batch_size]
                feed_dict={x: batch_x,y: batch_y}
                temp, c = sess.run([optimizer, cost], feed_dict)
                loss += c
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:', accuracy.eval({x: test_images, y:test_labels}))

def create_NN_model(classes, train_images, train_labels, test_images, test_labels,class_weights, hidden_units = 1500,number_of_hidden_units = 4,batch_size = 100,number_of_iterations = 20):
    x = tf.placeholder('float', [None, len(train_images[0])])
    y = tf.placeholder('float',[None, classes])
    NN_layers_units = []
    NN_layers_units.append(len(train_images[0]))
    for n in range(number_of_hidden_units):
        NN_layers_units.append(hidden_units)
    NN_layers_units.append(classes)
    train_neural_network(x,y,NN_layers_units,number_of_iterations,classes,train_images, train_labels, test_images, test_labels, batch_size,class_weights)

In [0]:
import os
import tarfile
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.utils import class_weight

def data_preprocess_kkanji(classes):
      x1 = []
      y1 = []

      number_of_files = []
      for folder in os.listdir('kkanji2'):
            number_of_files.append(len(os.listdir('kkanji2/'+folder)))
      number_of_files = sorted(number_of_files, reverse = True)
      val = number_of_files[classes]
      index1 = 0
      i = 0
      
      for folder in os.listdir('kkanji2'):
          if(len(os.listdir('kkanji2/'+folder)) > val):
            for file in os.listdir('kkanji2/'+folder):
                img=Image.open('kkanji2/'+folder+'/'+file)
                nomalized_image = np.array(img)/255
                y1.append(index1)
                x1.append(nomalized_image.reshape(nomalized_image.shape[0]*nomalized_image.shape[1]))
                img.close()
            index1+=1

      train_images, test_images, train_labels, test_labels = train_test_split(x1, y1, test_size=0.15, random_state=0, shuffle = True, stratify = y1)
      
      class_weights = class_weight.compute_class_weight('balanced', np.unique(train_labels),train_labels)

      train_images = np.array(train_images)
      train_labels = np.array(train_labels)
      test_images = np.array(test_images)
      test_labels = np.array(test_labels)

      sess = tf.Session()
      train_labels = sess.run(tf.one_hot(np.array(train_labels),classes))
      test_labels = sess.run(tf.one_hot(np.array(test_labels),classes))
      sess.close()
      return train_images, train_labels, test_images, test_labels,class_weights

classes = 30
train_images, train_labels, test_images, test_labels, class_weights = data_preprocess_kkanji(classes)

In [91]:
hidden_units = 1500
number_of_hidden_units = 4
batch_size = 100
number_of_iterations = 20

create_NN_model(classes, train_images, train_labels, test_images, test_labels,class_weights, hidden_units,number_of_hidden_units,batch_size,number_of_iterations)

Accuracy: 0.84139305


In [0]:

def data_preprocess_k49():
      classes = 49
      train_images = np.load('k49-train-imgs.npz')['arr_0']
      train_labels = np.load('k49-train-labels.npz')['arr_0']
      test_images = np.load('k49-test-imgs.npz')['arr_0']
      test_labels = np.load('k49-test-labels.npz')['arr_0']

      class_weights = class_weight.compute_class_weight('balanced', np.unique(train_labels),train_labels)
      train_images = train_images / 255.0
      test_images = test_images / 255.0

      x = tf.placeholder('float', [None, len(train_images[0])*len(train_images[0])])
      y = tf.placeholder('float',[None, classes])

      sess = tf.Session()
      train_images = np.array(train_images).reshape(len(train_images),len(train_images[0]) * len(train_images[0]))
      train_labels = sess.run(tf.one_hot(train_labels,classes))
      test_images = np.array(test_images).reshape(len(test_images),len(test_images[0]) * len(test_images[0]))
      test_labels = sess.run(tf.one_hot(test_labels,classes))
      sess.close()
      return train_images, train_labels, test_images, test_labels,class_weights

train_images, train_labels, test_images, test_labels,class_weights = data_preprocess_k49()

In [54]:
classes = 49
hidden_units = 1500
number_of_hidden_units = 4
batch_size = 100
number_of_iterations = 20

create_NN_model(classes, train_images, train_labels, test_images, test_labels,class_weights, hidden_units,number_of_hidden_units,batch_size,number_of_iterations)

Epoch 0 completed out of 20 loss: 3365300910.46875
Epoch 1 completed out of 20 loss: 699150604.8203125
Epoch 2 completed out of 20 loss: 369090364.46484375
Epoch 3 completed out of 20 loss: 230430987.76171875
Epoch 4 completed out of 20 loss: 171206752.20336914
Epoch 5 completed out of 20 loss: 118174574.94042969
Epoch 6 completed out of 20 loss: 95260966.03979492
Epoch 7 completed out of 20 loss: 75993976.12489319
Epoch 8 completed out of 20 loss: 60589653.88265991
Epoch 9 completed out of 20 loss: 54507414.18860245
Epoch 10 completed out of 20 loss: 47260893.67675173
Epoch 11 completed out of 20 loss: 40279340.329435825
Epoch 12 completed out of 20 loss: 38473414.762096405
Epoch 13 completed out of 20 loss: 34226559.517930984
Epoch 14 completed out of 20 loss: 33760288.6743927
Epoch 15 completed out of 20 loss: 28763907.132243156
Epoch 16 completed out of 20 loss: 28957125.530597687
Epoch 17 completed out of 20 loss: 25109377.592643738
Epoch 18 completed out of 20 loss: 29403007.4990

In [0]:
# Test your model using Kuzushiji-Kanji here

## Problem 2: Building a Decision Tree using Scikit-Learn's DecisionTreeClassifier

Repeat the above experiments using [Scikit-Learn's DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).

In [92]:
import numpy as np 
from sklearn import tree
from sklearn.metrics import accuracy_score
import tensorflow as tf
from sklearn.utils import class_weight

hidden_units = 512
batch_size = 100

train_images = np.load('k49-train-imgs.npz')['arr_0']
train_labels = np.load('k49-train-labels.npz')['arr_0']
test_images = np.load('k49-test-imgs.npz')['arr_0']
test_labels = np.load('k49-test-labels.npz')['arr_0']

class_weights = class_weight.compute_class_weight('balanced', np.unique(train_labels),train_labels)
feed_dict = {}
for i in range(0,len(class_weights)):
    feed_dict[i] = class_weights[i]

sess = tf.Session()
train_images = np.array(train_images).reshape(len(train_images),len(train_images[0]) * len(train_images[0]))
train_images = np.array(train_images).astype(float)
test_images = np.array(test_images).reshape(len(test_images),len(test_images[0]) * len(test_images[0]))
test_images = np.array(test_images).astype(float)
sess.close()


train_images = train_images / 255.0
test_images = test_images / 255.0

model = tree.DecisionTreeClassifier(class_weight = feed_dict)
model.fit(train_images, train_labels)
prediction = model.predict(test_images)

print(accuracy_score(test_labels, prediction)*100)

51.072716424105636


In [8]:
import numpy as np
from sklearn import tree
from sklearn.metrics import accuracy_score
import tensorflow as tf
from sklearn.utils import class_weight
import os
import tarfile
from PIL import Image
from sklearn.model_selection import train_test_split

classes = 30

x1 = []
y1 = []

number_of_files = []
for folder in os.listdir('kkanji2'):
    number_of_files.append(len(os.listdir('kkanji2/'+folder)))

number_of_files = sorted(number_of_files, reverse = True)
val = number_of_files[classes]

index1 = 0

for folder in os.listdir('kkanji2'):
    if(len(os.listdir('kkanji2/'+folder)) > val):
      for file in os.listdir('kkanji2/'+folder):
          img=Image.open('kkanji2/'+folder+'/'+file)
          normalized_image = np.array(img)/255
          y1.append(index1)
          x1.append(normalized_image.reshape(normalized_image.shape[0]*normalized_image.shape[1]))
          img.close()
      index1+=1

train_images, test_images, train_labels, test_labels = train_test_split(x1, y1, test_size=0.15, random_state=0, shuffle = True, stratify = y1)

class_weights = class_weight.compute_class_weight('balanced', np.unique(train_labels),train_labels)


train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

feed_dict = {}
for i in range(0,len(class_weights)):
    feed_dict[i] = class_weights[i]

model = tree.DecisionTreeClassifier(class_weight = feed_dict)

model.fit(train_images, train_labels)
prediction = model.predict(test_images)

print(accuracy_score(test_labels, prediction)*100)

68.45607808340728


## Problem 3: Building a Gradient Boosted Tree using Tensorflow's BoostedTreeClassifier estimator

In class we covered the vanilla decision tree. In recent years, a new kind of tree has become very popular in Kaggle competitions due to their performance, these trees are known as [Gradient Boosted Trees](https://en.wikipedia.org/wiki/Gradient_boosting#Gradient_tree_boosting). Typically, the winning submissions for most competitions make use of either some neural network or a gradient boosted tree (either, [XGBoost](https://xgboost.readthedocs.io/en/latest/) or [LightGBM](https://lightgbm.readthedocs.io/en/latest/)). For this problem, we will use [Tensorflow's BoostedTreeClassifier estimator](https://www.tensorflow.org/api_docs/python/tf/estimator/BoostedTreesClassifier). Repeat the above experiments using this model.

In [2]:
import lightgbm as lgb
import numpy as np 
import tensorflow as tf
from sklearn.metrics import accuracy_score 

hidden_units = 512
batch_size = 100

classes = 49

train_images = np.load('k49-train-imgs.npz')['arr_0']
train_labels = np.load('k49-train-labels.npz')['arr_0']
test_images = np.load('k49-test-imgs.npz')['arr_0']
test_labels = np.load('k49-test-labels.npz')['arr_0']

sess = tf.Session()
train_images = np.array(train_images).reshape(len(train_images),len(train_images[0]) * len(train_images[0]))
train_images = np.array(train_images).astype(float)
test_images = np.array(test_images).reshape(len(test_images),len(test_images[0]) * len(test_images[0]))
test_images = np.array(test_images).astype(float)
sess.close()

train_images = train_images / 255.0
test_images = test_images / 255.0

train_data = lgb.Dataset(train_images,label=train_labels)

params = dict()
params['boosting_type'] = 'gbdt'
params['application'] = 'multiclass'
params['metric'] = 'multi_logloss'
params['num_class'] = 49
params['is_unbalanced'] = 'true'
params['learning_rate'] = 0.05
params['num_leaves'] = 10
params['min_data_in_leaf'] = 1000
params['min_sum_hessian_in_leaf'] = 1e-3
params['max_depth'] = 10
params['max_bin'] = 256
params['feature_fraction'] = 0.7
params['bagging_fraction'] = 0.7
params['bagging_freq'] = 20

num_round=50
lgbm = lgb.train(params,train_data,num_round)
predictions=lgbm.predict(test_images)
result = []
for p in predictions:
    result.append(np.argmax(p))
accuracy_lgbm = accuracy_score(test_labels,result)
print(accuracy_lgbm)

0.5617817210159026


In [44]:
import lightgbm as lgb
import numpy as np 
import tensorflow as tf
from sklearn.metrics import accuracy_score 
import os
import tarfile
from PIL import Image


hidden_units = 512
batch_size = 100

x1 = []
y1 = []

classes = 30

i = 0

number_of_files = []
for folder in os.listdir('kkanji2'):
    number_of_files.append(len(os.listdir('kkanji2/'+folder)))

number_of_files = sorted(number_of_files, reverse = True)
val = number_of_files[classes]

index1 = 0

for folder in os.listdir('kkanji2'):
    if(len(os.listdir('kkanji2/'+folder)) > val):
      for file in os.listdir('kkanji2/'+folder):
          img=Image.open('kkanji2/'+folder+'/'+file)
          pix = np.array(img)/255
          y1.append(i)
          x1.append(pix.reshape(pix.shape[0]*pix.shape[1]))
          img.close()
      i+=1
      
train_images, test_images, train_labels, test_labels = train_test_split(x1, y1, test_size=0.15, random_state=0, shuffle = True, stratify = y1)
      
train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

train_data = lgb.Dataset(train_images,label=train_labels)

params = dict()
params['boosting_type'] = 'gbdt'
params['application'] = 'multiclass'
params['metric'] = 'multi_logloss'
params['num_class'] = classes
params['is_unbalanced'] = 'true'
params['learning_rate'] = 0.05
params['num_leaves'] = 10
params['min_data_in_leaf'] = 1000
params['min_sum_hessian_in_leaf'] = 1e-3
params['max_depth'] = 10
params['max_bin'] = 256
params['feature_fraction'] = 0.7
params['bagging_fraction'] = 0.7
params['bagging_freq'] = 20

num_round=50
lgbm = lgb.train(params,train_data,num_round)
predictions=lgbm.predict(test_images)
result = []
for p in predictions:
    result.append(np.argmax(p))
accuracy_lgbm = accuracy_score(test_labels,result)
print(accuracy_lgbm)

625
0.7888198757763976


In [0]:
#References

#(1) https://www.youtube.com/watch?v=yX8KuPZCAMo
#(2) https://towardsdatascience.com/methods-for-dealing-with-imbalanced-data-5b761be45a18
#(3) https://lightgbm.readthedocs.io/en/latest/